<h1>Use the GPU based environement otherwise the model will take around 1 hr for one image</h1>

In [1]:
!pip install diffusers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 6.7 MB/s eta 0:00:00


In [2]:
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 2.2 MB/s eta 0:00:00


In [3]:
from __future__ import annotations
import math
import random
import numpy as np
import torch
from PIL import Image
from diffusers import StableDiffusionXLPipeline, EulerAncestralDiscreteScheduler

import os
import random
import zipfile
from io import BytesIO

device = "cuda:0" # To run on cpu, comment this line
pipe = StableDiffusionXLPipeline.from_pretrained(
        "fluently/Fluently-XL-v4",
        torch_dtype=torch.float16,
        use_safetensors=True,
    )
pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)
pipe.load_lora_weights("ehristoforu/dalle-3-xl-v2", weight_name="dalle-3-xl-lora-v2.safetensors", adapter_name="dalle")
pipe.set_adapters("dalle")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

Fetching 20 files:   0%|          | 0/20 [00:00<?, ?it/s]

...

vae/config.json:   0%|          | 0.00/607 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/10.3G [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

dalle-3-xl-lora-v2.safetensors:   0%|          | 0.00/458M [00:00<?, ?B/s]

In [4]:
def generate_image(instruction: str = "Eiffel tower",
        steps: int = 8,
        randomize_seed: bool = False,
        seed: int = 25,
        width: int = 1024,
        height: int = 1024,
        guidance_scale: float = 6.2,
        use_resolution_binning: bool = True,
        save_path: str = ""):

    pipe.to(device)
    seed = int(randomize_seed_fn(seed, randomize_seed))
    generator = torch.Generator().manual_seed(seed)

    options = {
        "prompt":instruction,
        "width":width,
        "height":height,
        "guidance_scale":guidance_scale,
        "num_inference_steps":steps,
        "generator":generator,
        "use_resolution_binning":use_resolution_binning,
        "output_type":"pil",
    }

    output_image = pipe(**options).images[0]

    # for local save
    output_image.save("/content/new_img", format='PNG')

    print("image saving started")
    image_buffer = BytesIO()
    output_image.save(image_buffer, format='PNG')
    image_data = image_buffer.getvalue()

    # Uploading the images to the database
    # upload_image_to_db(image_data, save_path)
    print("Image saved successfully")

In [5]:
def randomize_seed_fn(seed, randomize_seed):
    if randomize_seed:
        return random.randint(0, 2**32 - 1)
    return seed

In [ ]:
generate_image(instruction="men wearning hat",
            steps = 8,
            randomize_seed = False,
            seed = 25,
            width = 1024,
            height = 1024,
            guidance_scale = 6.2,
            use_resolution_binning = True,
            save_path = f"new_img.jpg")


# generate_image(instruction=f"Generate an image of a {obj.age} {obj.gender} with {obj.ethinicty} {obj.skin_tone} {obj.body_type} {obj.hair_colour} wearing a {obj.clothing_top} and {obj.clothing_bottom} with a plain background",
#             steps = 8,
#             randomize_seed = False,
#             seed = 25,
#             width = 1024,
#             height = 1024,
#             guidance_scale = 6.2,
#             use_resolution_binning = True,
#             save_path = f"human_generated_images/inference_{obj.realtime_process_id}.jpg")

  0%|          | 0/8 [00:00<?, ?it/s]